In [1]:
import yfinance as yf
import random
import numpy as np
import pandas as pd
import math

# Define the stock ticker symbol
ticker_1 = 'AAPL'  # Example: Apple Inc.
ticker_2='MSFT'
ticker_3='GOOGL'
ticker_4='AMZN'

# Download historical data
data_AAPL = yf.download(ticker_1, start='2019-11-21', end='2022-12-31')
data_MSFT = yf.download(ticker_2, start='2019-11-21', end='2022-12-31')
data_GOOGL = yf.download(ticker_3, start='2019-11-21', end='2022-12-31')
data_AMZN = yf.download(ticker_4, start='2019-11-21', end='2022-12-31')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
data_AAPL['CODE']='AAPL'
data_MSFT['CODE']='MSFT'
data_GOOGL['CODE']='GOOGL'
data_AMZN['CODE']='AMZN'

In [3]:
import pandas_ta as ta
data_AAPL['RSI'] = ta.rsi(data_AAPL['Close'], length=14)
data_MSFT['RSI'] = ta.rsi(data_MSFT['Close'], length=14)
data_GOOGL['RSI'] = ta.rsi(data_GOOGL['Close'], length=14)
data_AMZN['RSI'] = ta.rsi(data_AMZN['Close'], length=14)
data_AAPL['macd'] = ta.macd(data_AAPL['Close'])['MACDh_12_26_9']
data_MSFT['macd'] = ta.macd(data_MSFT['Close'])['MACDh_12_26_9']
data_GOOGL['macd'] = ta.macd(data_GOOGL['Close'])['MACDh_12_26_9']
data_AMZN['macd'] = ta.macd(data_AMZN['Close'])['MACDh_12_26_9']
bb = ta.bbands(data_AAPL['Close'], length=20, std=2)
data_AAPL[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']] = bb[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']]
bb = ta.bbands(data_MSFT['Close'], length=20, std=2)
data_MSFT[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']] =bb[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']]
bb = ta.bbands(data_GOOGL['Close'], length=20, std=2)
data_GOOGL[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']] = bb[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']]
bb = ta.bbands(data_AMZN['Close'], length=20, std=2)
data_AMZN[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']] = bb[['BBU_20_2.0','BBM_20_2.0','BBL_20_2.0']]

In [4]:
data_AAPL=data_AAPL.loc['2020-01-21':]
data_MSFT=data_MSFT.loc['2020-01-21':]
data_GOOGL=data_GOOGL.loc['2020-01-21':]
data_AMZN=data_AMZN.loc['2020-01-21':]

In [5]:
from sklearn.model_selection import train_test_split
data_aa_70,data_aa_30=train_test_split(data_AAPL, test_size=0.3, shuffle=False)
data_ms_70,data_ms_30=train_test_split(data_MSFT, test_size=0.3, shuffle=False)
data_go_70,data_go_30=train_test_split(data_GOOGL, test_size=0.3, shuffle=False)
data_am_70,data_am_30=train_test_split(data_AMZN, test_size=0.3, shuffle=False)

In [6]:
import pandas as pd
data_train=pd.concat([data_aa_70,data_ms_70,data_go_70,data_am_70])
data_test=pd.concat([data_aa_30,data_ms_30,data_go_30,data_am_30])

In [7]:
def calculate_rsi_portfolio_values(data, initial_balance=10000/4):

    balance = initial_balance
    stock_quantity = 0
    portfolio_values = []
    prev_close_price=0

    for index, row in data.iterrows():
        close_price = row['Close']
        
        if index == 0:
            # First day: portfolio value is just the initial balance
            portfolio_value = initial_balance
            prev_close_price=close_price
        else:
            rsi = row['RSI']
            # Check RSI conditions for buy or sell
            if rsi < 30 and balance >= close_price:  # Buy condition
                shares_to_buy = balance // close_price  # Maximum shares we can afford
                balance -= shares_to_buy * close_price
                stock_quantity += shares_to_buy
                prev_close_price=close_price
            
            elif rsi > 75 and stock_quantity > 0:  # Sell condition
                balance += stock_quantity * close_price
                stock_quantity = 0
                prev_close_price=close_price
            else:
                balance+=stock_quantity*close_price-stock_quantity*prev_close_price
                prev_close_price=close_price
            
            # Calculate portfolio value
            portfolio_value = balance + (stock_quantity * close_price)
        
        portfolio_values.append(portfolio_value)
    return portfolio_values

def calculate_portfolio_values_macd_histogram(data, initial_balance=10000/4):

    balance = initial_balance
    stock_quantity = 0
    portfolio_values = []
    prev_close_price=0

    for index, row in data.iterrows():
        close_price = row['Close']
        
        if index == 0:
            portfolio_value = initial_balance
            prev_close_price=close_price
        else:
            macd_histogram = row['macd']
            # MACD Histogram strategy
            if macd_histogram > 1.4 and balance >= close_price:  # Buy
                shares_to_buy = balance // close_price
                balance -= shares_to_buy * close_price
                stock_quantity += shares_to_buy
                prev_close_price=close_price
            elif macd_histogram < -1 and stock_quantity > 0:  # Sell
                balance += stock_quantity * close_price
                stock_quantity = 0
                prev_close_price=close_price
            else:
                balance+=stock_quantity*close_price-stock_quantity*prev_close_price
                prev_close_price=close_price

            portfolio_value = balance + (stock_quantity * close_price)
        
        portfolio_values.append(portfolio_value)
    
    return portfolio_values


def calculate_portfolio_values_bollinger(data, initial_balance=10000/4):

    balance = initial_balance
    prev_close_price=0
    stock_quantity = 0
    portfolio_values = []

    for index, row in data.iterrows():
        close_price = row['Close']
        
        if index == 0:
            # First day's portfolio value is the initial balance
            portfolio_value = initial_balance
            prev_close_price=close_price
        else:
            bb_upper = row['BBU_20_2.0']
            bb_lower = row['BBL_20_2.0']

            # Bollinger Bands strategy
            if close_price <= bb_lower+16 and balance >= close_price:  # Buy
                shares_to_buy = balance // close_price
                balance -= shares_to_buy * close_price
                stock_quantity += shares_to_buy
                prev_close_price=close_price
            elif close_price >= bb_upper-20 and stock_quantity > 0:  # Sell
                balance += stock_quantity * close_price
                stock_quantity = 0
                prev_close_price=close_price
            else:
                balance+=stock_quantity*close_price-stock_quantity*prev_close_price
                prev_close_price=close_price

            portfolio_value = balance + (stock_quantity * close_price)
        
        portfolio_values.append(portfolio_value)
    
    return portfolio_values




In [8]:
stocks=['AAPL','MSFT','GOOGL','AMZN']
init_balance=10000
each_stock=10000/4
RSI_portfolio=[]
MACD_portfolio=[]
BB_portfolio=[]
for stock in stocks:
        RSI_portfolio +=calculate_rsi_portfolio_values(data_train[data_train.CODE==stock], 10000/4)
        MACD_portfolio += calculate_portfolio_values_macd_histogram(data_train[data_train.CODE==stock], 10000/4)
        BB_portfolio +=calculate_portfolio_values_bollinger(data_train[data_train.CODE==stock], 10000/4)


In [9]:
data_train['RSI_portfolio']=RSI_portfolio
data_train['MACD_portfolio']=MACD_portfolio
data_train['BB_portfolio']=BB_portfolio


In [10]:
RSI_portfolio=[]
MACD_portfolio=[]
BB_portfolio=[]
for stock in stocks:
        RSI_portfolio +=calculate_rsi_portfolio_values(data_test[data_test.CODE==stock], 10000/4)
        MACD_portfolio += calculate_portfolio_values_macd_histogram(data_test[data_test.CODE==stock], 10000/4)
        BB_portfolio +=calculate_portfolio_values_bollinger(data_test[data_test.CODE==stock], 10000/4)

In [11]:
data_test['RSI_portfolio']=RSI_portfolio
data_test['MACD_portfolio']=MACD_portfolio
data_test['BB_portfolio']=BB_portfolio

In [12]:
data_train.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CODE', 'RSI',
       'macd', 'BBU_20_2.0', 'BBM_20_2.0', 'BBL_20_2.0', 'RSI_portfolio',
       'MACD_portfolio', 'BB_portfolio'],
      dtype='object')

In [13]:
test_columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CODE', 'RSI',
       'macd', 'BBU_20_2.0', 'BBM_20_2.0', 'BBL_20_2.0']
train_columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CODE', 'RSI',
       'macd', 'BBU_20_2.0', 'BBM_20_2.0', 'BBL_20_2.0',
       'RSI_portfolio', 'MACD_portfolio', 'BB_portfolio']

In [14]:
data_train.reset_index(drop=True,inplace=True)
data_test.reset_index(drop=True,inplace=True)

In [15]:
def calculate_sharpe_ratio(portfolio_returns, risk_free_rate=0.01):
    if len(portfolio_returns) == 0:
        return np.nan
    excess_returns = portfolio_returns - risk_free_rate
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns) if np.std(excess_returns) != 0 else np.nan
    return min(max(sharpe_ratio, -5), 5)  # Cap values to avoid outliers

# Function to calculate Maximum Drawdown
def calculate_max_drawdown(portfolio_values):
    if len(portfolio_values) == 0:
        return np.nan
    cumulative_max = portfolio_values.cummax()
    drawdowns = (portfolio_values - cumulative_max) / cumulative_max
    return drawdowns.min()  # Cap at -90% drawdown

# Function to calculate Total Return
def calculate_total_return(start_value, end_value, dividends=0):
    total_return = (end_value - start_value + dividends) / start_value
    return min(max(total_return, -0.9), 5)  # Cap total return between -90% and 500%

# Function to calculate Annualized Return
def calculate_annualized_return(start_value, end_value, n_years):
    if n_years > 0:
        annualized_return = (end_value / start_value) ** (1 / n_years) - 1
        return min(max(annualized_return, -0.9), 5)  # Cap annualized return
    return np.nan

# Function to calculate Annualized Volatility
def calculate_annualized_volatility(portfolio_returns, periods_per_year=252):
    return min(np.std(portfolio_returns) * np.sqrt(periods_per_year), 5) 

In [16]:
portfolio_frsl=[]

In [17]:
import gym
from gym import spaces
import numpy as np

# pass normal data to train in it
class FinancialMarketEnv(gym.Env):
    def __init__(self, data, data_not_scaled ,code,strategies=['RSI','MACD','BollingerBands'],strategy_code=[0,1,2],stock_codes=['AAPL','MSFT','GOOGL','AMZN'] ,initial_balance=10000):
        super(FinancialMarketEnv, self).__init__()
        self.data = data
        self.data_not_scaled=data_not_scaled
        self.code=code
        self.ENVIRONMENT_SHAPE = (len(train_columns),)
        self.initial_balance = initial_balance
        self.cash_stocks={s: initial_balance / len(stock_codes) for s in stock_codes}
        self.max_steps = len(data)
        self.current_step = 0
        self.current_code='a'
        self.strategies=strategies
        self.strategies_code=strategy_code
        self.balance = initial_balance
        self.portfolio = {code:0}
        self.action_space = spaces.Discrete(3)  # 0: Sell, 1: Hold, 2: Buy
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(len(data.columns)-1,), dtype=np.float32)
        self.prev_property=self.initial_balance
        self.current_prop=self.initial_balance
        

    def reset(self):
        self.current_step = 0
        self.current_date=0
        self.balance = self.initial_balance
        self.portfolio = {self.code:0}
        self.cash_stocks[self.code]=self.initial_balance/4
        return self.data_not_scaled.iloc[self.current_step].values

    def _get_observation(self):
        return self.data_not_scaled.iloc[self.current_step-1].values

    # code means aapl and others etc.......
    
    def step(self, action):
        strategy = self.strategies[action]
        prev_balance=self.balance+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
        reward = self._execute_strategy(strategy)
        self.current_step += 1
        done = self.current_step >= self.max_steps
        if self.current_step % 25 == 0:  # Check if the step is a multiple of 25
            print(f"Step: {self.current_step}")
            print(f"Currnt balance: {self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']}")
        portfolio_frsl.append(self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close'])
        if self.current_step==1:
            print('your money in start of episode:  ' ,self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close'])
        elif self.current_step == self.max_steps:
            print('Your money in end of  episode : ' , self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close'],'Next episode starting .......')
        if self.balance+self.portfolio.get(self.code,0)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']<10100:
            reward = -reward
        return self._get_observation(), reward, done, {}

    
    def _execute_strategy(self, strategy ):
        # Determine the action (buy/sell/hold) based on strategy
        pp=0
        if strategy == 'RSI':
            action_type = self._get_rsi_action(self.code)
            pp=0
        elif strategy == 'MACD':
            action_type = self._get_macd_action(self.code)
            pp=1
        elif strategy == 'BollingerBands':
            action_type = self._get_bollinger_action(self.code)
            pp=2
        
        if action_type == 'buy':
            if self.cash_stocks[self.code]/self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close'] < 1:
                return 0
            amount_to_invest = self.cash_stocks[self.code]
            shares_to_buy = int(amount_to_invest / self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close'])
            if self.balance > amount_to_invest:
                 self.balance -= int(shares_to_buy*0.8)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
                 self.cash_stocks[self.code] -= int(shares_to_buy*0.8)*self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
                 self.portfolio[self.code] = self.portfolio.get(self.code,0)+int(shares_to_buy*0.8)
                 return self.reward_fast(pp,self.code)  # Reward can be profit/loss here
            else:
                return 0
        elif action_type == 'sell':
            if self.portfolio.get(self.code,0)==0:
                return 0
            amount_to_sell = self.portfolio.get(self.code,0)
            self.portfolio[self.code] -= int(amount_to_sell*0.8)
            self.balance += int(amount_to_sell*0.8) * self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
            self.cash_stocks[self.code] += int(amount_to_sell*0.8) * self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
            return self.reward_fast(pp,self.code)
        elif action_type=='hold' and self.current_step > 1:
            return  (self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']-self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-2]['Close'])/self.data_not_scaled[self.data_not_scaled.CODE==self.code].iloc[self.current_step-1]['Close']
        else:
            return 0# No transaction

    def calculate_reward(self,metrics): # code = aapl andothers

     metric_ranges = {
    'Sharpe_Ratio': {'min': -2, 'max': 3},  # Typical range [-2, 3]
    'Max_Drawdown': {'min': -1, 'max': 0},  # Range [-1, 0]
    'Total_Return': {'min': -1, 'max': 2},  # Assumes [-100%, +200%]
    'Annualized_Return': {'min': -1, 'max': 1},  # Assumes [-100%, +100%]
    'Annualized_Volatility': {'min': 0, 'max': 0.5}  # Assumes [0%, 50%]
     }

# Normalize each metric using min-max normalization
     normalized_metrics = {
     key: (value - metric_ranges[key]['min']) / (metric_ranges[key]['max'] - metric_ranges[key]['min'])
     for key, value in metrics.items()}
   

     weights = {
        'Sharpe_Ratio': 0.7,
        'Max_Drawdown': 0.25,
        'Total_Return': 0.2,
        'Annualized_Return': 0.15,
        'Annualized_Volatility': 0.1
        }
     reward = (
            weights['Sharpe_Ratio'] * normalized_metrics['Sharpe_Ratio'] -
            weights['Max_Drawdown'] * normalized_metrics['Max_Drawdown'] +
            weights['Total_Return'] * normalized_metrics['Total_Return'] +
            weights['Annualized_Return'] * normalized_metrics['Annualized_Return'] -
            weights['Annualized_Volatility'] * normalized_metrics['Annualized_Volatility']
        )
     
     return reward

    def reward_fast(self, pp, code):
       risk_free_rate = 0.01
       periods_per_year = 252

       # Filter the dataset for the given stock code and relevant columns
       df = self.data_not_scaled[self.data_not_scaled.CODE == code].loc[:self.current_step, 
                                               ['RSI_portfolio', 'MACD_portfolio', 'BB_portfolio']]
    
       step_rewards = {}
       rewards = []

    # Iterate through the columns (strategies)
       for strategy in df.columns:
        # Portfolio values and returns calculation
           portfolio_values = df[strategy][:self.current_step]
           portfolio_returns = portfolio_values.pct_change().dropna()
           portfolio_values_a=np.array(portfolio_values)
           start_value = portfolio_values_a[0]
           end_value = portfolio_values_a[-1]
           n_years = (self.current_step + 1) / periods_per_year

        # Compute metrics for the given strategy
           metrics = {
               'Sharpe_Ratio': calculate_sharpe_ratio(portfolio_returns, risk_free_rate),
               'Max_Drawdown': calculate_max_drawdown(portfolio_values),
               'Total_Return': calculate_total_return(start_value, end_value),
               'Annualized_Return': calculate_annualized_return(start_value, end_value, n_years),
               'Annualized_Volatility': calculate_annualized_volatility(portfolio_returns, periods_per_year)
           }

        # Calculate the reward based on metrics
           reward = self.calculate_reward(metrics)
           rewards.append(reward)

    # Ensure rewards list is cleaned of NaNs
       rewards = [0 if math.isnan(x) else x for x in rewards]
       return rewards[pp]

    
    def _get_rsi_action(self,code):
        if self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['RSI']>70 :
            return 'sell'
        elif self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['RSI']<30 :
            return 'buy'
        else:
            return 'hold'
    
    def _get_macd_action(self,code):
        if self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['macd']>0 and self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-2]['macd']<0:
            return 'buy'
        elif self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['macd']<0 and self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-2]['macd']>0:
            return 'sell'
        else:
            return 'hold'
    
    def _get_bollinger_action(self,code):
        if self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['Close']>=self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['BBU_20_2.0']:
            return 'sell'
        elif self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['Close']<=self.data_not_scaled[self.data_not_scaled.CODE==code].iloc[self.current_step-1]['BBL_20_2.0']:
            return 'buy'
        else:
            return 'hold'

    def render(self, mode='human'):
        print(f"Step: {self.current_step}, Balance: {self.balance}, Portfolio: {self.portfolio}")

    def close(self):
     print("Environment closed.")

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
class fsrl:
    def __init__(self, state_size, action_size, gamma=0.99, learning_rate=0.01,
                 epsilon_start=1.0, epsilon_min=0.05, epsilon_decay=0.995,
                 memory_size=10000, batch_size=64, target_update_freq=30):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.replay_memory = deque(maxlen=memory_size)
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.target_update_counter = 0
        self.model_action=[]


        # Create main and target networks
        self.model = self.build_network()
        self.target_model = self.build_network()
        self.target_model.set_weights(self.model.get_weights())

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    def build_network(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_dim=14),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='softmax')
        ])
        return model

    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done) # yaad rakhlena bro ####################################
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    # Trains main network every step during episode
    def train(self, terminal_state, step):
        # Start training only if certain number of samples is already saved
        MIN_REPLAY_MEMORY_SIZE=250
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE: # ################### need to figure out this number 
            return

        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, self.batch_size)

        # Get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0] for transition in minibatch])
        current_states = np.delete(current_states, 6, axis=1)
        current_states = current_states.astype(np.float32)
    
        # Remove the 7th column (index 6) from current_states
        current_qs_list = self.model.predict(current_states)
        scaler = MinMaxScaler(feature_range=(0 , 1))  # Change to (-1, 1) if needed
        current_qs_list = scaler.fit_transform(current_qs_list)
        
        

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array([transition[3] for transition in minibatch])
        new_current_states = np.delete(new_current_states, 6, axis=1)
        new_current_states = new_current_states.astype(np.float32)
        future_qs_list = self.target_model.predict(new_current_states) # ask once what to use here
        scaler = MinMaxScaler(feature_range=(0 , 1))  # Change to (-1, 1) if needed
        future_qs_list = scaler.fit_transform(future_qs_list)
        

        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + self.gamma * max_future_q  # BELLMAN EQUATION
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)
            
        
        X = np.array([np.delete(row, 6) for row in X],dtype=np.float32)
        y= np.array(y,dtype=np.float32)
        self.model.compile(
            optimizer=self.optimizer,
            loss= self.loss_function ,
            metrics=['accuracy']
        )
        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(X,y,verbose=0)
        return

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self,state):
        return self.model.predict(state)

    def select_action(self, state):
        state=np.delete(state, 6)
        q_values = self.model.predict(np.array(state[np.newaxis, :],dtype=np.float32))
        return np.argmax(q_values[0])

    def train_current(self,env,episodes,max_steps):
        for episode in range(episodes):
            state = env.reset()
            total_reward = 0
            action=1

            for step in range(max_steps):
                num=np.random.random()
                if num > self.epsilon:
                # Get action from Q network 
                    action = self.select_action(state)
                    self.model_action.append(action)
                    print('model predicted : ',action)

                else:
                # Get random action 
                    action = random.choice([0, 1, 2])
                    print('random choice : ',action)
                next_state, reward, done, _ = env.step(action)
                #reward=reward/10
                if num > self.epsilon and len(self.model_action)>=3 and action==self.model_action[-1] and action==self.model_action[-2] and action==self.model_action[-3]:
                    reward=-reward
                self.update_replay_memory([state, action, reward, next_state, done])
                state = next_state
                total_reward += reward

                self.train(done, step)
                self.target_update_counter += 1
                if self.target_update_counter > self.target_update_freq :
                    self.target_model.set_weights(self.model.get_weights())
                    self.target_update_counter = 0
                if self.epsilon > self.epsilon_min:
                    self.epsilon *= self.epsilon_decay

                if done:
                    break

    def backtest(self,states):
        results=self.target_model.predict(states[1])
        print(results)
        action=np.argmax(results)
        print(action)
        action_type='a'
        if action == 0 :
            action_type = self._get_rsi_action(states)
        elif action == 1 :
            action_type = self._get_macd_action(states)
        elif action == 2 :
            action_type = self._get_bollinger_action(states)
        return action_type
        
        

    
    def _get_rsi_action(self,states):
        print(states[1,0,6])
        if states[1,0,6]>70 :
            return 'sell'
        elif states[1,0,6]<30 :
            return 'buy'
        else:
            return 'hold'
    
    def _get_macd_action(self,states):
        print(states[1,0,7])
        if states[1,0,7]>0 and states[0,0,7]<0:
            return 'buy'
        elif states[1,0,7]<0 and states[0,0,7]>0:
            return 'sell'
        else:
            return 'hold'
    
    def _get_bollinger_action(self,states):
        if states[1,0,3]>=states[1,0,8]:
            return 'sell'
        elif states[1,0,3]<=states[1,0,10]:
            return 'buy'
        else:
            return 'hold'

In [20]:
from collections import deque
import tensorflow as tf

In [21]:
model=fsrl((len(train_columns)-1),3)

C:\Users\Jatin Sharma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
hello=data_train[data_train.CODE=='AAPL']

In [23]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assuming `hello` is your DataFrame
scaler = MinMaxScaler()

# Scale the numeric columns to [0, 1]
numeric_columns = hello.select_dtypes(include=['float64', 'int64']).columns
hello.loc[:, numeric_columns] = scaler.fit_transform(hello[numeric_columns])

# Now shift the scaled values to be in the range (0, 1]
hello.loc[:, numeric_columns] = 0.2 + (hello[numeric_columns] * 0.9)

C:\Users\Jatin Sharma\AppData\Local\Temp\ipykernel_23720\3575491028.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.18117092 0.1577972  0.16464424 0.27376099 0.3137143  0.31447693
 0.45453866 0.22241498 0.41137299 0.34444865 0.24802573 0.20188011
 0.16711784 0.19891572 0.17729449 0.13831859 0.18866022 0.13941947
 0.10145937 0.28930819 0.13743872 0.1545122  0.2297061  0.47001427
 0.49200695 0.40739488 0.72528754 1.         0.77922025 0.72235636
 0.46218879 0.38020492 0.4803424  0.63745376 0.63368006 0.55666208
 0.97915489 0.85531374 0.73000337 0.73423776 0.67244326 0.59883583
 0.93562605 0.76717284 0.63949366 0.68118077 0.54755311 0.4233789
 0.32937252 0.40466395 0.35075199 0.3240746  0.23055174 0.41716272
 0.41992996 0.33175586 0.31344142 0.23351612 0.39945734 0.23385749
 0.30122487 0.45199865 0.23090244 0.36313351 0.19678452 0.21741174
 0.22090529 0.19736868 0.18418407 0.24974916 0.36850925 0.51779928
 0.24

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

exclude_columns = [3,6, 7, 8, 9, 10, 11,12,13,14]

# Create a copy of the DataFrame to normalize
df_normalized = data_train[data_train.CODE=='AAPL'].copy()

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize each row
for index, row in data_train[data_train.CODE=='AAPL'].iterrows():
    # Get column names to normalize
    cols_to_normalize = [col for col in data_train[data_train.CODE=='AAPL'].columns if data_train[data_train.CODE=='AAPL'].columns.get_loc(col) not in exclude_columns]
    
    # Reshape the row for MinMaxScaler
    values_to_scale = row[cols_to_normalize].values.reshape(-1, 1)
    
    # Scale the values and update the DataFrame
    df_normalized.loc[index, cols_to_normalize] = scaler.fit_transform(values_to_scale).flatten()



C:\Users\Jatin Sharma\AppData\Local\Temp\ipykernel_23720\647694129.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9999999999999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_normalized.loc[index, cols_to_normalize] = scaler.fit_transform(values_to_scale).flatten()


In [25]:
envo_aapl=FinancialMarketEnv(hello,df_normalized,'AAPL')

In [26]:
model.train_current(envo_aapl,2,len(data_train[data_train.CODE=='AAPL'])) # already trained so now just loading the model

random choice :  2
your money in start of episode:   2500.0
random choice :  2
random choice :  1
random choice :  0
random choice :  0
random choice :  1
random choice :  1
random choice :  1
random choice :  0
random choice :  1
random choice :  0
random choice :  1
random choice :  1
random choice :  1
random choice :  2
random choice :  0
random choice :  1
random choice :  0
random choice :  2
random choice :  1
random choice :  1
random choice :  0
random choice :  1
random choice :  2
random choice :  2
Step: 25
Currnt balance: 2434.3499603271484
random choice :  0
random choice :  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
model predicted :  2
random choice :  1
random choice :  2
random choice :  1
random choice :  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
model predicted :  2
random choice :  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
model predicted :  2
random choice :  1
random choice :  0
random choice :  1
random choice :  1
random choice :  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/s

### So portfolio value of frsl model at end = 4205-2500=1705 profit

In [ ]:
#need to add that model cant predict more than 4 same strategy

In [27]:
!pip install stable-baselines3



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
!pip install shimmy
!pip install gymnasium


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import gym
from gym import spaces
import numpy as np

# pass normal data to train in it
class FinancialMarketEnvout(gym.Env):
    def __init__(self, data, data_not_scaled ,code,strategies=['RSI','MACD','BollingerBands'],strategy_code=[0,1,2],stock_codes=['AAPL','MSFT','GOOGL','AMZN'] ,initial_balance=10000):
        super(FinancialMarketEnvout, self).__init__()
        self.data = data
        self.data_not_scaled=data_not_scaled
        self.stock_codes=stock_codes
        self.code=code
        self.ENVIRONMENT_SHAPE = (len(train_columns),)
        self.initial_balance = initial_balance
        self.cash_stocks={s: initial_balance / len(stock_codes) for s in stock_codes}
        self.max_steps = len(data)
        self.current_step = 0
        self.current_code='a'
        self.strategies=strategies
        self.strategies_code=strategy_code
        self.balance = initial_balance
        self.portfolio = {code:0}
        self.action_space = spaces.Discrete(3)  # 0: buy, 1: sell, 2: hold 
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(len(data.columns)-1,), dtype=np.float32)
        self.prev_property=self.initial_balance
        self.current_prop=self.initial_balance

    def reset(self):
        self.current_step = 0
        self.cash_stocks={s: self.initial_balance / len(self.stock_codes) for s in self.stock_codes}
        self.current_date=0
        self.balance = self.initial_balance
        self.portfolio = {self.code:0}
        return self.data_not_scaled.iloc[self.current_step].values

    def _get_observation(self):
        return self.data_not_scaled.iloc[self.current_step-1].values

    # code means aapl and others etc.......
    
    def step(self, action):
        if self.current_step==0:
            print('your money in start of episode:  ' ,self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close'])
        reward = self._execute_strategy(action)
        self.current_step += 1
        done = self.current_step >= self.max_steps
        if self.current_step % 50 == 0:  # Check if the step is a multiple of 50
            print(f"Step: {self.current_step}")
            print(f"Currnt balance: {self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close']}")
       
        elif self.current_step == self.max_steps:
            print('Your money in end of  episode : ' , self.cash_stocks[self.code]+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close'],'Next episode starting .......')
        if self.balance+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close']<10100:
            reward = -reward
        return self._get_observation(), reward, done, {}

    def _execute_strategy(self, action ):
        # Determine the action (buy/sell/hold) based on strategy
        
        
        if action == 0 : ## 0 means buy 
            if self.cash_stocks[self.code]/self.data_not_scaled.iloc[self.current_step-1]['Close'] < 1:
                return 0
            amount_to_invest = self.cash_stocks[self.code]
            shares_to_buy = int(amount_to_invest / self.data_not_scaled.iloc[self.current_step-1]['Close'])
            if self.balance > amount_to_invest:
                 self.balance -= int(shares_to_buy*0.8)*self.data_not_scaled.iloc[self.current_step-1]['Close']
                 self.cash_stocks[self.code] -= int(shares_to_buy*0.8)*self.data_not_scaled.iloc[self.current_step-1]['Close']
                 self.portfolio[self.code] = self.portfolio.get(self.code,0)+int(shares_to_buy*0.8)
                 return self.balance+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close']-(self.initial_balance/4) # Reward can be profit/loss here
            else:
                return 0
        elif action == 1 :  ## 1 means sell
            if self.portfolio.get(self.code,0)==0:
                return 0
            amount_to_sell = self.portfolio.get(self.code,0)
            self.portfolio[self.code] -= int(amount_to_sell*0.8)
            self.balance += int(amount_to_sell*0.8) * self.data_not_scaled.iloc[self.current_step-1]['Close']
            self.cash_stocks[self.code] += int(amount_to_sell*0.8) * self.data_not_scaled.iloc[self.current_step-1]['Close']
            return self.balance+self.portfolio.get(self.code,0)*self.data_not_scaled.iloc[self.current_step-1]['Close']-(self.initial_balance/4)
        else:
            return 0  # No transaction



    def render(self, mode='human'):
        print(f"Step: {self.current_step}, Balance: {self.balance}, Portfolio: {self.portfolio}")

    def close(self):
     print("Environment closed.")

In [28]:
import gym
from stable_baselines3 import PPO
#from stable_baselines3.common.envs import DummyVecEnv

# Create your environment (for example, a custom trading environment)
# Ensure that your environment follows the OpenAI Gym interface (reset, step, observation_space, action_space, etc.)
da=data_train[data_train.CODE=='AAPL'].drop(columns=['CODE'])
env = FinancialMarketEnvout(hello,da,'AAPL')  # Replace with your custom environment, e.g., 'TradingEnv'

# Wrap the environment in a vectorized environment (required by Stable Baselines3)

# Create the PPO model
model_ppo = PPO("MlpPolicy", env, verbose=1)

# Train the PPO model (this will take some time based on your environment)
model_ppo.learn(total_timesteps=520)  # Adjust total_timesteps as needed

# Save the trained model
model_ppo.save("ppo_trained_model.zip")

# Test the trained model
obs = env.reset()
done = False
while not done:
    action, _states = model_ppo.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()  # Optionally render the environment if needed

print("Model training complete and saved!")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Jatin Sharma\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


your money in start of episode:   2500.0
Step: 50
Currnt balance: 2192.3324127197266
Step: 100
Currnt balance: 2514.017509460449
Step: 150
Currnt balance: 2891.3648834228516
Step: 200
Currnt balance: 2410.1301498413086
Step: 250
Currnt balance: 2714.5502166748047
Step: 300
Currnt balance: 2976.600425720215
Step: 350
Currnt balance: 3101.4702529907227
Step: 400
Currnt balance: 3340.6704177856445
Step: 450
Currnt balance: 3289.100471496582
Step: 500
Currnt balance: 3338.130485534668
Your money in end of  episode :  3238.13045501709 Next episode starting .......
your money in start of episode:   2500.0
Step: 50
Currnt balance: 2435.1824226379395
Step: 100
Currnt balance: 2771.836982727051
Step: 150
Currnt balance: 3054.084518432617
Step: 200
Currnt balance: 2734.3096771240234
Step: 250
Currnt balance: 2828.7397537231445
Step: 300
Currnt balance: 2697.609664916992
Step: 350
Currnt balance: 2634.889602661133
Step: 400
Currnt balance: 2822.0498428344727
Step: 450
Currnt balance: 2902.5696792

### So portfolio value at end incase of ppo model is 11532-10000=1532 profit 

In [30]:
import gym
from stable_baselines3 import DQN

# Create your custom trading environment
# Ensure that your environment follows the OpenAI Gym interface (reset, step, observation_space, action_space, etc.)
da = data_train[data_train.CODE == 'AAPL'].drop(columns=['CODE'])
env = FinancialMarketEnvout(hello, da, 'AAPL')  # Replace with your custom environment, e.g., 'TradingEnv'

# Create the DQN model
model_dqn = DQN("MlpPolicy", env, verbose=1)

# Train the DQN model (this will take some time based on your environment)
model_dqn.learn(total_timesteps=520)  # Adjust total_timesteps as needed

# Save the trained model
model_dqn.save("dqn_trained_model.zip")

# Test the trained model
obs = env.reset()
done = False
while not done:
    action, _states = model_dqn.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()  # Optionally render the environment if needed

print("DQN model training complete and saved!")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
your money in start of episode:   2500.0
Step: 50
Currnt balance: 2546.1926345825195
Step: 100
Currnt balance: 3167.2200355529785


C:\Users\Jatin Sharma\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Step: 150
Currnt balance: 3554.922618865967
Step: 200
Currnt balance: 3268.062572479248
Step: 250
Currnt balance: 3748.3226203918457
Step: 300
Currnt balance: 3457.452495574951
Step: 350
Currnt balance: 3376.2525672912598
Step: 400
Currnt balance: 3637.06254196167
Step: 450
Currnt balance: 3560.1425132751465
Step: 500
Currnt balance: 4120.492519378662
Your money in end of  episode :  4145.0924644470215 Next episode starting .......
your money in start of episode:   2500.0
Step: 1, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 2, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 3, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 4, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 5, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 6, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 7, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 8, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 9, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 10, Balance: 10000, Portfolio: {'AAPL': 0}
Step: 11, Balance: 10000, Portfolio: {'AAP

#### Better than both  

#### Tried for google and got better result than both
##### But couldnt shocase it here as showing storage error while running for both the stocks 

In [57]:
class Backtest(gym.Env):
    def __init__(self, backtest_data):  # Corrected method name
        super(Backtest, self).__init__()

        self.backtest_data = backtest_data
        self.current_step = 0
        self.current_step_model = 1
        self.initial_balance = 2500  # Initial portfolio value
        self.balance = self.initial_balance
        self.stocks_held = 0
        self.stock_price_my_model = self.backtest_data.iloc[1, 3]
        self.portfolio_value = 2500

        self.action_space = spaces.Discrete(3)  # Action space: 0=buy, 1=hold, 2=sell
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(14,), dtype=np.float32)

    def reset_my_model(self):
        self.current_step_model = 1
        self.balance = self.initial_balance
        self.stocks_held = 0
        self.stock_price_my_model = self.backtest_data.iloc[self.current_step_model,3]
        self.portfolio_value=2500
        return self.my_model_get_state()

    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.stocks_held = 0
        self.stock_price = self.backtest_data.iloc[self.current_step,3]
        self.portfolio_value=2500
        return self._get_state()

    def step_my_model(self,action):
        if self.current_step_model==1:
            print('your initial portfolio is : ',self.portfolio_value)
        self.stock_price_my_model = self.backtest_data.iloc[self.current_step_model,3]
        if action == 'buy' :  # Buy
            amount_to_buy = self.balance // self.stock_price_my_model
            self.stocks_held += amount_to_buy
            self.balance -= amount_to_buy * self.stock_price_my_model
        elif action == 'sell' :  # Sell
            self.balance += self.stocks_held * self.stock_price_my_model
            self.stocks_held = 0
        else:
            pass
        
        self.portfolio_value = self.balance + self.stocks_held * self.stock_price_my_model
        print(self.portfolio_value,action)
        self.current_step_model += 1
        done = self.current_step_model >= len(self.backtest_data)
        if done:
            print('your final  portfolio is : ',self.portfolio_value)
        return self.my_model_get_state(), done, {}

    def step(self, action):
        # Retrieve the current stock price
        if self.current_step==0:
            print('your initial portfolio is : ',self.portfolio_value)
        self.stock_price = self.backtest_data.iloc[self.current_step,3]
        # Take action: Buy, hold, or sell
        if action == 0 :  # Buy
            amount_to_buy = self.balance // self.stock_price
            self.stocks_held += amount_to_buy
            self.balance -= amount_to_buy * self.stock_price
        elif action == 1:  # Sell
            self.balance += self.stocks_held * self.stock_price
            self.stocks_held = 0
        else:
            pass

        # Calculate the portfolio value (balance + value of stocks held)
        self.portfolio_value = self.balance + self.stocks_held * self.stock_price

        # Move to the next step
        self.current_step += 1
        done = self.current_step >= len(self.backtest_data)
        if done:
            print('your final  portfolio is : ',self.portfolio_value)

        # Return the next state, done, and info
        return self._get_state(), done, {}

    def _get_state(self):
        # Generate the state as a 14-dimensional vector: [Open, High, Low, Portfolio value, ...]
        row = self.backtest_data.iloc[self.current_step,:]
        row = np.array(row).reshape(1, 14)  # Shape becomes (1, 14)

        return np.array(row)

    def my_model_get_state(self):
        row = self.backtest_data.iloc[self.current_step_model-1:self.current_step_model+1,:]
        row = np.array(row).reshape(2, 1, 14)
        return row
        

    def render(self):
        print(f"Balance: {self.balance}, Stocks Held: {self.stocks_held}, Stock Price: {self.stock_price}")

In [58]:
da = data_train[data_train.CODE == 'AAPL'].drop(columns=['CODE'])
print(da.shape)
backtest_model=Backtest(da)
backtest_model_ppo=Backtest(da)
backtest_model_dqn=Backtest(da)
# Reset the environment to start a new episode
obs_model = backtest_model.reset_my_model()
obs_model_ppo = backtest_model_ppo.reset()
obs_model_dqn = backtest_model_dqn.reset()

(520, 14)


In [59]:
import pickle

# Load the trained model
with open('frsl_trained_model.pkl', 'rb') as f:
    model = pickle.load(f)  # Load the frsl object

# Now, you can use frsl_loaded just like the original frsl object

from stable_baselines3 import PPO, DQN

# Load the PPO model
model_ppo = PPO.load("ppo_trained_model.zip")

# Load the DQN model
model_dqn = DQN.load("dqn_trained_model.zip")

C:\Users\Jatin Sharma\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [60]:
done = False
i=0
while not done:
    # Predict the action to take
    action = model.backtest(obs_model)  # deterministic=True for greedy policy

    # Step the environment forward with the chosen action
    obs_model, done, info = backtest_model.step_my_model(action)


    # Optionally, render the environment to visualize the interaction

print("Prediction complete!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
[[7477550.5 6216124.  8598700. ]]
2
your initial portfolio is :  2500
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[[7671307. 6377151. 8821541.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[[10759060.  8943293. 12372785.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[11889645.  9882875. 13673074.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[[11911248.  9900855. 13697908.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[15874689. 13194761. 18256274.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[ 9306111.  7735801. 10701719.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[14653186. 12179596. 16851424.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[12759372. 10605702. 14673337.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
[[10030829.  8338092. 11535227.]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[ 8725019.   7252876.5 10033404. ]]
2
2500.0 hold
1/1 ━━━━━━━━━━━━━━━

ValueError: cannot reshape array of size 14 into shape (2,1,14)

#### FRSL BACKTEST FINAL PORTFOLIO = 3097
#### PPO BACKTEST FINAL PORTFOLIO = 2834
#### DQN BACKTEST FINAL PORTFOLIO = 2719

# FRSL MACHAYA !!!!!

#### Frsl also performed better in google but cant show as not able to run both as memory issue . 